# Homework 3

Letizia Russo, Daniel Losada Molina and Michele Luca Puzzo

In [ ]:
pip install nltk

Note: you may need to restart the kernel to use updated packages.


In [1]:
import requests
import lxml
import shutil
#import functions as fun
import urllib.request
import urllib.error as uer
import time
import nltk
import datetime
import csv
import os

from tqdm import tqdm
from bs4 import BeautifulSoup
from pathlib import Path
from nltk.corpus import stopwords
from nltk.stem.snowball import PorterStemmer
from nltk.tokenize import RegexpTokenizer

In [2]:
def link_list(anime):
    with open('urls.txt', 'w') as file:
        for page in range(0, anime+50, 50):
            url = 'https://myanimelist.net/topanime.php?limit=' + str(page)
            response = requests.get(url)
            soup = BeautifulSoup(response.content)
            ranking_list = soup.find_all(class_='ranking-list')
            for l in ranking_list:
                link = l.find_all(class_='detail')[0].a['href']
                file.write("%s\n" % link)

In [8]:
nFolders = 299

months = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun','Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']

def crawl(url_file):
    Path("directory").mkdir(exist_ok=True)
    for i in range(1,nFolders+1):
        nomeCartella = 'cartelle/cartella{}'.format(i)
        Path(nomeCartella).mkdir(exist_ok=True)
    num_lines = sum(1 for line in open(url_file, encoding="utf-8"))
    pagesPerFolder = num_lines/nFolders
    
    c = 0
    lastFolder = 1
    
    with open(url_file, "r", encoding="utf-8") as a:
        for i in range(c):
            print("skipping url: ", i)
            next(a)
        for i in a:
            page = requests.get(i)
            soup = BeautifulSoup(page.content, features ="lxml")
            number = int(c/(pagesPerFolder)) + 1
            if number != lastFolder:
                print("I'm waiting")
                lastFolder = number
                time.sleep(200)            
            print("Going to save in cartella" + str(number) + ", the page " + str(c+1))
            f = open("./cartelle/cartella{}".format(number) + "/page_{}.html".format(c+1), "w",encoding="utf-8")
            f.write(soup.prettify())
            f.close()

            file_size = os.path.getsize("./cartelle/cartella{}/page_{}.html".format(number, c + 1))
            if file_size < 50000:
                os.remove("./cartelle/cartella{}/page_{}.html".format(number, c + 1))
                page = requests.get(i)
                soup = BeautifulSoup(page.content, features ="lxml")
                f = open("./cartelle/cartella{}".format(number) + "/page_{}.html".format(c+1), "w",encoding="utf-8")
                f.write(soup.prettify())
                f.close()

            else:
                c = c + 1

def findField(array, word):
    for x in array:
        #print("#####################################")
        #print(" ".join((x.find("span").text).split()))
        #print("#####################################")
        if " ".join((x.find("span").text).split()) == word:
            return x
    return -1       
        
        
def cleaner(text):
    stop_words = set(stopwords.words('english')) # retrieve stop words
    tokenizer = RegexpTokenizer("[\w']+") # recognize the tokens
    parole = tokenizer.tokenize(text) # tokenize the text
    correct_words = [] # save the correct words to consider like tokens
    for word in parole:
        # check if the word is lower and it isn't a stop word or a number
        if word.lower() not in stop_words and word.isalpha(): 
            word = PorterStemmer().stem(word) # use the stemmer function
            correct_words.append(word.lower()) # insert the good token to lower case
        
    return correct_words

def create_vocabulary(data):
    ### Input == I use like input the dataset obtain in exercise 1 where i apply the clean text function
    ### Output == I obtain a vocabulary, the keys are all tokens (with no repeat) contained in the synopsis for the each rows
    ### for each token I define the index of the rows where the token is in the synopsis
    voc = {}
    for i, row in data.iterrows():
            if len(data.at[i, "Synopsis"]) > 0:  # check if the list is empty or not to avoid the eventually error
                for word in data.at[i, "Synopsis"]: # bring the token from the list
                    if word in voc.keys(): # insert the token into the vocabulary with the documents where this is present
                        if i not in voc[word]:
                            voc[word].append(i)
                    else:
                        voc[word] = [i]
    return voc

def extractData(pagePath):
    with open(pagePath, encoding="utf-8") as fp:
        soup = BeautifulSoup(fp, "html.parser")
        animeTitle = " ".join((soup.html.head.title.text).split())
        
        #print("animeTitle: ", animeTitle)
        animeInfo = (soup.find_all("div",  attrs={ "class" : "spaceit_pad"}))
        #print("Animeinfo: ", animeInfo)
        
        animeType = " ".join((findField(animeInfo, "Type:").a.text).split())
        #print("animeType :", animeType)
        
        animeNumEpisode = int(list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Episodes:").findAll(text=True, recursive=False)) )))[0])
        #print("animeNumEpisode: ", animeNumEpisode)
        
        aired = list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Aired:").findAll(text=True, recursive=False)) )))[0].split('to')
        releaseDate = datetime.datetime(int(aired[0].split()[2]), months.index(aired[0].split()[0])+1 , int(aired[0].split()[1][:1]))
        if len(aired) == 2 and len(aired[0].split()) == 3 and len(aired[1].split()) == 3:
            endDate = datetime.datetime(int(aired[1].split()[2]), months.index(aired[1].split()[0])+1 , int(aired[1].split()[1][:1]))
        else:
            endDate = ""
        #print("releaseDate: ", releaseDate)
        #print("endDate: ", endDate)
        
        animeNumMembers = int(list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Members:").findAll(text=True, recursive=False)) )))[0].replace(",", ""))
        #print("animeNumMembers: ", animeNumMembers)
        
        animeScore = float(" ".join((soup.find_all("span", itemprop = "ratingValue")[0].text).split()))
        #print("animeScore: ", animeScore)
        
        animeUsers = int(" ".join((soup.find_all("span", itemprop = "ratingCount")[0].text).split()))
        #print("animeUsers: ", animeUsers)
        
        animeRank = int(list(filter(lambda x: x[0] == '#',findField(animeInfo, "Ranked:").text.split()))[0].replace('#',''))
        #print("animeRank: ", animeRank)

        animePopularity = int(list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Popularity:").findAll(text=True, recursive=False)) )))[0].replace("#", ""))
        #print("animePopularity: ", animePopularity)
        
        animeDescription = soup.find_all("p", itemprop = "description")[0].text.strip().replace('\n', '').replace('  ', '')
        #print("animeDescription: ", animeDescription)
        
        animeRelatedAHref = soup.find(name="table",attrs={"class":"anime_detail_related_anime"}).findChildren('a', href=True)
        animeRelated = []
        for x in animeRelatedAHref:
            aux = " ".join((x.text).split())
            if aux not in animeRelated:
                animeRelated.append(aux)
        #print("animeRelated: ", animeRelated)
        
        animeCharactersRaw = soup.find_all("h3", attrs={"class" : "h3_characters_voice_actors"})
        animeCharacters = []
        for x in animeCharactersRaw:
            animeCharacters.append(" ".join((x.text).split()))
        #print("animeCharacters: ", animeCharacters)
        
        animeVoicesRaw = soup.find_all("td", attrs={"class" : "va-t ar pl4 pr4"})
        animeVoices = []
        for x in animeVoicesRaw:
            animeVoices.append(" ".join((x.contents[1].text).split()))
        #print("animeVoices: ", animeVoices)
        
        animeStaff = ""
        if(len(soup.find_all("div", attrs={"class" : "detail-characters-list clearfix"})) == 2):
            aux = soup.find_all("div", attrs={"class" : "detail-characters-list clearfix"})[1]
            animeStaffRaw = []
            for x in aux.findChildren('a'):
                if (" ".join((x.text).split()) != ''):
                    animeStaffRaw.append(" ".join((x.text).split()))
            animeStaffTaskRaw = []
            for x in aux.findChildren('small'):
                animeStaffTaskRaw.append(" ".join((x.text).split()))
            #print("animeStaffRaw: ", animeStaffRaw)
            #print("animeStaffTaskRaw: ", animeStaffTaskRaw)
            animeStaff = [list(a) for a in zip(animeStaffRaw, animeStaffTaskRaw)]

        
        #print("animeStaff: ", animeStaff)
        
        return [animeTitle, animeType, animeNumEpisode, releaseDate, endDate, animeNumMembers, animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, animeCharacters, animeVoices, animeStaff]
    
    
    
def extractData2(pagePath):
    with open(pagePath, encoding="utf-8") as fp:
        soup = BeautifulSoup(fp, "html.parser")
        
        try:
            animeTitle = " ".join((soup.html.head.title.text.replace("- MyAnimeList.net", '')).split())
        except:
            animeTitle = " "
        #print("animeTitle: ", animeTitle)
        
        try:
            animeInfo = (soup.find_all("div",  attrs={ "class" : "spaceit_pad"}))
        #print("Animeinfo: ", animeInfo)
        except:
            animeInfo = " "
        
        try:
            animeType = " ".join((findField(animeInfo, "Type:").a.text).split())
        #print("animeType :", animeType)
        except:
            animeType = " "
        try:
            animeNumEpisode = int(list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Episodes:").findAll(text=True, recursive=False)) )))[0])
        #print("animeNumEpisode: ", animeNumEpisode)
        except:
            animeNumEpisode = " "
        
        try:
            aired = list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Aired:").findAll(text=True, recursive=False)) )))[0].split('to')
            releaseDate = datetime.datetime(int(aired[0].split()[2]), months.index(aired[0].split()[0])+1 , int(aired[0].split()[1][:1]))
            if len(aired) == 2:
                endDate = datetime.datetime(int(aired[1].split()[2]), months.index(aired[1].split()[0])+1 , int(aired[1].split()[1][:1]))
            else:
                endDate = ""
        except:
            releaseDate = " "
            endDate = " "
        #print("releaseDate: ", releaseDate)
        #print("endDate: ", endDate)
        
        try:
            animeNumMembers = int(list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Members:").findAll(text=True, recursive=False)) )))[0].replace(",", ""))
        #print("animeNumMembers: ", animeNumMembers)
        except:
            animeNumMembers = " "
        
        try:
            animeScore = float(" ".join((soup.find_all("span", itemprop = "ratingValue")[0].text).split()))
        #print("animeScore: ", animeScore)
        except:
            animeScore = " "

        try:
            animeUsers = int(" ".join((soup.find_all("span", itemprop = "ratingCount")[0].text).split()))
        #print("animeUsers: ", animeUsers)
        except:
            animeUsers = " "
        
        try:
            animeRank = int(list(filter(lambda x: x[0] == '#',findField(animeInfo, "Ranked:").text.split()))[0].replace('#',''))
        #print("animeRank: ", animeRank)
        except:
            animeRank = " "
        
        try:
            animePopularity = int(list(filter(lambda y: y != '',list(map(lambda x: " ".join((x).split()),findField(animeInfo, "Popularity:").findAll(text=True, recursive=False)) )))[0].replace("#", ""))
        #print("animePopularity: ", animePopularity)
        except:
            animePopularity = " "
        
        try:
            animeDescription = soup.find_all("p", itemprop = "description")[0].text.strip().replace('\n', '').replace('  ', '')
        #print("animeDescription: ", animeDescription)
        except:
            animeDescription = " "
        
        try:
            animeRelatedAHref = soup.find(name="table",attrs={"class":"anime_detail_related_anime"}).findChildren('a', href=True)
            animeRelated = []
            for x in animeRelatedAHref:
                aux = " ".join((x.text).split())
            if aux not in animeRelated:
                animeRelated.append(aux)
        #print("animeRelated: ", animeRelated)
        except:
            animeRelated = " "
        
        try:
            animeCharactersRaw = soup.find_all("h3", attrs={"class" : "h3_characters_voice_actors"})
            animeCharacters = []
            for x in animeCharactersRaw:
                animeCharacters.append(" ".join((x.text).split()))
        #print("animeCharacters: ", animeCharacters)
        except:
            animeCharacters = " "

        try:
            animeVoicesRaw = soup.find_all("td", attrs={"class" : "va-t ar pl4 pr4"})
            animeVoices = []
            for x in animeVoicesRaw:
                animeVoices.append(" ".join((x.contents[1].text).split()))
        #print("animeVoices: ", animeVoices)
        except:
            animeVoices = " "


        try:
            animeStaff = ""
            if(len(soup.find_all("div", attrs={"class" : "detail-characters-list clearfix"})) == 2):
                aux = soup.find_all("div", attrs={"class" : "detail-characters-list clearfix"})[1]
                animeStaffRaw = []
            for x in aux.findChildren('a'):
                if (" ".join((x.text).split()) != ''):
                    animeStaffRaw.append(" ".join((x.text).split()))
            animeStaffTaskRaw = []
            for x in aux.findChildren('small'):
                animeStaffTaskRaw.append(" ".join((x.text).split()))
            #print("animeStaffRaw: ", animeStaffRaw)
            #print("animeStaffTaskRaw: ", animeStaffTaskRaw)
            animeStaff = [list(a) for a in zip(animeStaffRaw, animeStaffTaskRaw)]
        except:
            animeStaff = " "

        
        #print("animeStaff: ", animeStaff)
        
        return [animeTitle, animeType, animeNumEpisode, releaseDate, endDate, animeNumMembers, animeScore, animeUsers, animeRank, animePopularity, animeDescription, animeRelated, animeCharacters, animeVoices, animeStaff]

def tsvGenerator():
    c = 83
    url_file = 'urls.txt'
    num_lines = sum(1 for line in open(url_file))
    pagesPerFolder = num_lines/nFolders
    outputTSV = "dataset.tsv"

    with open(outputTSV, 'wt') as out_file:
        tsv_writer = csv.writer(out_file, delimiter='\t')
        tsv_writer.writerow(["animeTitle", "animeType", "animeNumEpisode", "releaseDate", "endDate", "animeNumMembers", "animeScore", "animeUsers", "animeRank", "animePopularity", "animeDescription", "animeRelated", "animeCharacters", "animeVoices", "animeStaff"])
        for x in range(1, 20000 +1):

            number = int(c/(pagesPerFolder)) + 1
            path = "./cartella{}".format(number) + "/page_{}.html".format(c+1)
            print("path: ", path)
            tsv_writer.writerow(extractData(path))
            c = c + 1

def getUrl(lineNum, url_file):
    with open(url_file) as fp:
        for i, line in enumerate(fp):
            if i == lineNum-1:
                return line
            elif i >= lineNum:
                break

def downloadOneFile(pageNum):
    
    num_lines = sum(1 for line in open("urls.txt"))
    pagesPerFolder = num_lines/nFolders
    url = getUrl(pageNum, "urls.txt")
    page = requests.get(url)
    soup = BeautifulSoup(page.content, features ="lxml")
    cartellaNumber = int(pageNum/(pagesPerFolder)) + 1
    print("Cartella" + str(cartellaNumber) + ".  Page_"+ str(pageNum))
    f = open("./cartella{}".format(cartellaNumber) + "/page_{}.html".format(pageNum), "w",encoding="utf-8")
    f.write(soup.prettify())
    f.close()

# Data Collection

Since no dataset was provided for this homework, we had to create our own dataset by scraping the given website, which was done in the following steps.

## 1.1 Get the list of animes

Firstly, we had to get the list of all the anime we wanted to add to our dataset. This included all the anime found in the first 400 pages. Therefore, we created a function that scrapes a certain number of pages from an initial url, to extract the url of each of them. The initial url and the following number of pages are given as input, while the resulting urls are stored in a txt file called "urls.txt" .

In [ ]:
link_list(20000)     


100%|██████████| 20000/20000 [00:00<00:00, 950345.87it/s]


## 1.2 Crawl books

In [7]:
crawl("urls.txt")

skipping url:  0
skipping url:  1
skipping url:  2
skipping url:  3
skipping url:  4
skipping url:  5
Going to save in cartella1, the page 7
Going to save in cartella1, the page 8
Going to save in cartella1, the page 9
Going to save in cartella1, the page 10


KeyboardInterrupt: 

In [12]:
tsvGenerator()

path:  ./cartella2/page_84.html
path:  ./cartella2/page_85.html
path:  ./cartella2/page_86.html
path:  ./cartella2/page_87.html
path:  ./cartella2/page_88.html
path:  ./cartella2/page_89.html
path:  ./cartella2/page_90.html
path:  ./cartella2/page_91.html
path:  ./cartella2/page_92.html
path:  ./cartella2/page_93.html
path:  ./cartella2/page_94.html
path:  ./cartella2/page_95.html
path:  ./cartella2/page_96.html
path:  ./cartella2/page_97.html
path:  ./cartella2/page_98.html
path:  ./cartella2/page_99.html
path:  ./cartella2/page_100.html
path:  ./cartella3/page_101.html
path:  ./cartella3/page_102.html
path:  ./cartella3/page_103.html
path:  ./cartella3/page_104.html
path:  ./cartella3/page_105.html
path:  ./cartella3/page_106.html
path:  ./cartella3/page_107.html
path:  ./cartella3/page_108.html
path:  ./cartella3/page_109.html
path:  ./cartella3/page_110.html
path:  ./cartella3/page_111.html
path:  ./cartella3/page_112.html
path:  ./cartella3/page_113.html
path:  ./cartella3/page_11

AttributeError: 'int' object has no attribute 'a'

In [ ]:
def clean_text(text):
    stop_words = set(stopwords.words('english')) # obtain the stop words
    good_words = [] # save the correct words to consider like tokens
    tokenizer = RegexpTokenizer("[\w']+") # function to recognize the tokens
    words = tokenizer.tokenize(text) # tokenize the text 
    for word in words:
        # check if the word is lower and it isn't a stop word or a number
        if word.lower() not in stop_words and word.isalpha(): 
            word = PorterStemmer().stem(word) # use the stemmer function
            good_words.append(word.lower()) # insert the good token to lower case
        
    return good_words

In [ ]:
a = "I'm Michele, and I live in Ostia"

In [ ]:
clean_text(a)

['michel', 'live', 'ostia']

In [ ]:
import nltk
nltk.download()

showing info https://raw.githubusercontent.com/nltk/nltk_data/gh-pages/index.xml
